# Bienvenida

¡Hola! Gracias por entrar, ¿ya te serviste tu tacita de café? Si no te gusta el café, igual puedes hacerte un tecito o servirte una cerveza, not judging; al fin y al cabo, vivimos una pandemia y yo solo busco entretenerte un rato. Antes de empezar, quiero aclarar algo: según las estadísticas de Kaggle, llegaste a este post por estas posibilidades:
1. Tenemos una amistad o una relación de aprecio, tal que entraste a un post de Data Science (DS o Ciencia de Datos) por qué en verdad me estimas <3.
2. Eres mi coach de Platzi (Hola César).
3. Eres un curioso de la Ciencia de Datos e hispanohablante.
4. Lo mismo que el anterior, pero quieres practicar tu español.
5. Lo viste en twitter y te ganó la curiosidad.

Cómo puedes observar (si eres de los últimos tres sub-grupos) el post NO está dirigido a un público técnico del área, pero no por ello demerita la predicción. Todo lo contrario, busco hacer un post amigable para todo el público, ya que los resultados deberían ser de dominio público. Si en verdad buscamos reducir el impacto que tiene esta enfermedad, todos debemos de ser conscientes de la importancia de respetar la cuarentena **HASTA QUE SE HAYAN ACABADO LOS BROTES**.

Sí, la situación es fea, pero dado el resultado... Espero que el modelo haya exagerado la predicción. Sobre esto, lo entenderán al final.

## Import

No-DS: Estas son las herramientas y bibliotecas que use para realizar las predicciones. Se tienen que colocar "a fuerza" (no-mexicanos: significa 'obligatoriamente') para que se realicen los cálculos. ¿Qué quiere decir esto en lenguaje común? Que cuando se actualice la información, sin que yo "le pique a la computadora" los datos se actualizarán, mostrando así que mi opinión personal o el cesgo que pueda tener el día de hoy es completamente irrelavante (te hablo a tí, troll de twitter). Esto igual permite que cualquiera pueda copiar lo que ve aquí, "pegarlo en su programa dónde hace programas" y le funcionaría. De no tener una sola de estas cosas, el programa no funcionaría. En este caso en particular, le puse que fuera opcional que se mostrará el código. Ya saben, para no marearlos :)

In [ ]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta, date
import calendar

#Manage of files
import pandas as pd
import csv
import numpy as np

#Graph tools
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import matplotlib.patches as mpatches

#interactive visualization
import plotly.express as px
import plotly.graph_objs as go

#Gauss-Jordan
import math

## Loading Info

Como es posible de apreciar, la información que uso se encuentra a disposición de todos. Tan así, que tú puedes descargar "el excel" que usé y hacer la predicción por tu cuenta. Se encuentra al final del post y solo debes de buscar el que termine como dice aquí. Puedes realizar cualquier modelo que te imagines y créeme, te acordarás de tu profe de álgebra de la prepa o secundaria.

Estas son las últimas líneas de código que dejaré forzosamente visibles para todos. Ya que el post se encuentra enfocado a gente que no tiene buenas bases de mates, pero tiene (aunque no creas en ti, yo sí lo hago <3).

In [ ]:
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
df.dtypes

## Edition del DataFrame (los datos, para los cuates)

Lo que obervan arriba, son los datos que contiene el set de datos. Seamos honestos y respondamos esta pregunta: ¿en verdad nos pondremos a interpretar toda la información de cada estado/provincia que existe en el mundo? Por qué digo, al menos en México, la gente de Monterrey sigue peleando en que no son provincia.

Para este trabajo en concreto, que es un reto personal, puesto por mi coach, que tiene por objetivo mejorar mis habilidades con el análisis de datos, interpretación y presentación, no. Por ello, me limitaré a la información que hay en el mundo de forma general y (spoiler alert) haré una predicción para mi país (México, por si mi nombre les confundió). Se analizarán datos acumulados (que el día de mañana incluye los de mañana y el acumulado de hoy), y solo lo registrado de forma **oficial** como COVID-19 según el gobierno de cada país (al menos, eso comenta el creador de la fuente original). Los datos son:

1. Casos confirmados de contagio.
1. Muertes confirmadas.
1. Casos de recuperación.

In [ ]:
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])
df = df[['ObservationDate',
         'Country/Region',
         'Confirmed',
         'Deaths',
         'Recovered']]

In [ ]:
#Hola extraño, si te preguntas que es esto, es para hacer que los colores hagan match con el estilo. Sí, TOC
colors = [u'#348ABD', u'#A60628', u'#7A68A6', u'#467821', u'#D55E00', u'#CC79A7', u'#56B4E9', u'#009E73', u'#F0E442', u'#0072B2']
names = ['blue', 'red', 'purple', 'green', 'orange', 'pink', 'light_blue', 'intense_green', 'pastel_yellow', 'dark_blue']
bmh_colors = dict(zip(names, colors))


x = df['ObservationDate']
y_df = ['Confirmed', 'Deaths', 'Recovered']
titles = ['COVID-19 Confirmed Cases by date', 'COVID-19 Deaths Cases by date', 'COVID-19 Recovered Cases by date']
colors = [bmh_colors['blue'], 'k', bmh_colors['green']]

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5), sharey=False, sharex=True)
plt.style.use('ggplot')
n = 0

for ax in fig.get_axes():
    ax.scatter(x, df[ y_df[n] ], s=1, c=colors[n])
    ax.set_title(titles[n])

    date_format = mdates.DateFormatter('%m')
    ax.set_xlabel('Dates')
    ax.xaxis.set_major_formatter(date_format)

    n = n + 1

plt.show()

# Análisis Mundial (General)

Como puedes observar, la información es demasiado variada. Se pueden apreciar demasiadas tendencias, líneas que se cortan y algunas que se desfasan. Por ello, se hace un "filtro" en el que se suman TODOS los casos de los 3 eventos, por separado. Hay que recordar que la información se encontraba a nivel provincia/estado; entonces, si se sumamos los datos de todos los estados, de un país por día, nos daría la cifra nacional (adelanto del final).

## Días mostrados.

Para el momento que se hizo este análisis (21 de agosto de 2020), la información disponible llegaba hasta el día 13 del mismo mes, con países sin información de algunos estados. Por dicho motivo, se decidió segmentar la información hasta el día anterior. Ya que es posible la edición a posteriori, me reservaré de forma temporal la oportunidad de editarlo después de la revisión de mi Coach :).

In [ ]:
start_day = datetime(year=2020, month=1, day=22)
end_day = datetime(year=2020, month=8, day=12)
days = end_day - start_day
n_days = days.days

list_days = [start_day + timedelta(days=x) for x in range(n_days)]

In [ ]:
dates = df['ObservationDate'].to_list()
confirmed = df['Confirmed'].to_list()
deaths = df['Deaths'].to_list()
recovered = df['Recovered'].to_list()
countries = df['Country/Region'].to_list()

In [ ]:
sum_confirmed = [0] * (n_days)
sum_deaths = [0] * (n_days)
sum_recovered = [0] * (n_days)

day_subregion = [None] * (n_days)
day_continent = [None] * (n_days)

plus = 0
for d in range(len(list_days)):

    while dates[plus] == list_days[d] and (d + plus) < len(dates):
        '''---BY DAY---'''
        #General
        sum_confirmed[d] = int(sum_confirmed[d]) + int(confirmed[d + plus])
        sum_deaths[d] = int(sum_deaths[d]) + int(deaths[d + plus])
        sum_recovered[d] = int(sum_recovered[d]) + int(recovered[d + plus])
        
        plus += 1

In [ ]:
ys = [sum_confirmed, sum_deaths, sum_recovered]
titles = ['COVID-19 Confirmed Cases by date', 'COVID-19 Deaths Cases by date', 'COVID-19 Recovered Cases by date']
colors = [bmh_colors['blue'], 'k', bmh_colors['green']]

In [ ]:
x = list_days
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5), sharey=False, sharex=True)
plt.style.use('ggplot')
    
n = 0

for ax in fig.get_axes():
    ax.scatter(x, ys[n], s=1, color=colors[n])
    ax.set_title(titles[n])

    date_format = mdates.DateFormatter('%m')
    ax.set_xlabel('Dates')
    ax.xaxis.set_major_formatter(date_format)

    n = n + 1

plt.show()

# Predicciones (matemáticas, no soy vidente).

Durante la prepa y parte de la carrera (ingeniería) existe una materia que a muchos les aterra: álgebra. Y cuando la logras pasar, existe una peor: modelos estadísticos o numéricos. En lo personal, a mi me encantaban, pero entiendo por qué no era un sentimiento general ese amor a las mates. Al final, si todos fueramos matemáticos, el mundo sería un bellísimo lugar.

¿Por qué esta intro si ya existía otra? Verás pequeño Timmy (o pequeña Karen), a lo largo de mi vida como asesor de estas materías, este es el tema que siempre les costaba entender dada la forma en que se enseñaba: ¡Vamos directo al procedimiento! Queridos docentes: no hagan eso. Afortunadamente, ahora tenemos tecnología y Kaggle, lo que nos permite mostrar a nuestros alumnos la universalidad de las metemáticas, y por qué es importante saber que existe un método para nuestra situación, así cómo leer el procedimiento.

Por esto, aquí verán los modelos que NO ENCAJARON con los datos. Usualmente, la gente que no entiende de esto cree que solo es picarle a un botón que diga "do math" y no comprende lo que significa "el modelo no se ajusta a los datos". Tan tedioso puede llegar a ser, que se nos hizo preferible crear la Inteligencia Artificial (estadística avanzada hecha por computadoras) para no repetir el proceso infernal otra vez. En resumen y recordándoles, este post busca estimular el pensamiento matemático con un tema de relevancia real. ¿Quieres saber cuando saldrás?, ¿no entiendes por qué te regañan por ir a un restaurante o a una fiesta con tus amigos? Sigue leyendo, por qué te mostraré por que las mates y el poder del virus son mas reales que la promesa de ser tu propio jefe a través de dos aplicaciones.

## Regresión Lineal

El primer módelo visto por todos. Tan poderoso y útil para nubes sin forma  como para datos "sin sentido" pero agrupados. Afortunadamente, el buen Ricardo Celis tiene un excelente curso en platzi sobre este tema; y si no cuentas con platzi, igual está el canal en Youtube *Dot CSV (esta en español y con animaciones geniales)*. En ambos casos, encontrarás una explicación distinta y complementaria de lo que es.
* El profe Celis explica el proceso, y de donde sale.
* Dot CSV busca crear interés en los modelos de Inteligencia Artificial (gracias) a través de ejemplos apantalladores.

Resumen del modelo: "Trazamos" una línea **RECTA** que indique la tendencia que tienen los datos, el camino que podrían recorrer, y se puede complementar con la varianza para obtener un rango esperado. Este rango es donde PODRÍAN caer los datos, de seguir la misma tendencia. En este caso, no use la varianza más que usando "un marcador" más grueso para la tendencia, mientras que con los datos use "un bolígrado de punta fina".

In [ ]:
def estimate_b0_b1(x, y):
    n = np.size(x)

    #Obtain average of x and y
    mean_x, mean_y = np.mean(x), np.mean(y)

    #Calculate sumatory of XY and XX
    sum_xy = np.sum((x - mean_x)*(y - mean_y))
    sum_xx = np.sum((x*(x - mean_x)))

    #Regresion coeficents
    b_1 = sum_xy / sum_xx
    b_0 = mean_y - b_1*mean_x

    b = [b_0, b_1]

    return b

def linear_regression(x, y):
    if type(x[0]) == datetime:
        x = map(datetime.toordinal, x)
        x = list(x)

    b = estimate_b0_b1(x, y)

    y_pred = [b[0] + b[1]*x[x_i] for x_i in range(len(x))]

    #Graficación
    plot_regression(x, y,y_pred)


def plot_regression(x, y, y_pred, x_labels=None, y_labels=None, titles=None):
    fig, ax = plt.subplots(figsize=(7,5))
    plt.style.use('ggplot')
    
    plt1 = ax.scatter(x, y, label='Data', color = 'g', marker='o', s=10)

    try:
        plt2 = sns.regplot(x, y_pred, label='Linear Regresion Model', 
                        scatter_kws={'s':1},
                        line_kws={"color":"r","alpha":0.3,"lw":10})
        
    except:
        plt.cla()
        plt1 = ax.scatter(x, y, label='Data', color = 'g', marker='o', s=10)
        plt3 = ax.plot(x, y_pred, label='Linear Regresion Model', color ='b')
        

    #X axis format
    date_format = mdates.DateFormatter('%m')
    ax.set_xlabel('Dates')
    ax.xaxis.set_major_formatter(date_format)

    #Labels
    if x_labels == None:
        ax.set_xlabel('2020 months')
    else:
        pass

    if y_labels == None:  
        ax.set_ylabel('Confirmed Cases / 10,000,000')
    else:
        pass

    if titles == None: 
        ax.set_title('COVID-19 Confirmed Cases')
    
    plt.legend()

    plt.show()

In [ ]:
x = list_days
y = ys[0]

linear_regression(x,y)

Como es posible apreciar, tenemos "dos tendencias": Una al inicio de la pandemia (solo China y algunos países europeos, **aparentemente**) y otra dónde se ve el boom mundial. Por ello, se decidió hacer un filtro donde la tendencia mundial ya estaba "encaminada". Por ello, de forma personal, se escoge en abril y buscamos la posición que se encuentra el primer día de abril. Si tú, quieres usar otra fecha en tu modelo, adelante.

In [ ]:
april_pos = 0
while list_days[april_pos].month < 4:
    april_pos += 1

april_pos

In [ ]:
x = list_days[70:]
y = ys[0][70:]

linear_regression(x,y)

Aquí, podemos ver como la línea del modelo se acerca más hacia la recta. Solo que el caracter de curva que tiene hace que nunca volvamos a tocarla. Haciendo un experimento social, algunas amistades me comentarón que parecía "exponencial", para ellas y ellos, el siguiente modelo.

## Regresión Exponencial (a.k.a. "la curveada hacia arriba").

Usualmente, mis amistades no matemáticas cuando piensan en una línea curveada hacia arriba, creen que es un comportamiento exponencial, por qué en la carrera siempre les mostraron una curva de ingresos con ese comportamiento (les invito a googlearlas para que se den una idea de por qué ese ejemplo)


Y... pues sí, es una curva... va hacia arriba... Pinta bien, ¿no? Vamos a probar el modelo entonces. 

In [ ]:
def obtain_alpha_beta(x, y):
    #Iterables
    x_2 = [x[i]**2 for i in range(len(x))]
    log_y = [math.log(y[i], 10) for i in range(len(y))]
    x_log_y = [x[i]*log_y[i] for i in range(len(log_y))]
    n = len(x)

    #Summatories
    sum_x = sum(x)
    sum_x_2 = sum(x_2)

    sum_log_y = sum(log_y)
    sum_x_log_y = sum(x_log_y)

    #ALPHA

    #Matrix Solver
    #Numerador
    Mn_00_11_alpha = sum_log_y * sum_x_2
    Mn_01_10_alpha = sum_x_log_y * sum_x
    #Denominador
    Md_00_11 = n * sum_x_2
    Md_01_10 = sum_x * sum_x

    numerador_alpha = Mn_00_11_alpha - Mn_01_10_alpha
    denominador = Md_00_11 - Md_01_10
    log_alpha = numerador_alpha/denominador

    #BETHA
    #Numerador
    Mn_00_11_betha = n * sum_log_y
    Mn_01_10_betha = sum_x * sum_x_log_y

    numerador_betha = Mn_00_11_betha - Mn_01_10_betha
    log_betha = numerador_betha/denominador


    alpha = 10**log_alpha
    betha = 10**log_betha

    return alpha, betha, log_alpha, log_betha

def exponencial_regression(x, y):
    if type(x[0]) == datetime:
        ordinal_x = map(datetime.toordinal, x)
        ordinal_x = list(ordinal_x)
    else:
        ordinal_x = x

    alpha, betha, log_alpha, log_betha = obtain_alpha_beta(ordinal_x, y)

    y_pred = [alpha*betha**ordinal_x[x_i] for x_i in range(len(ordinal_x))]

    #Graficación
    plot_regression(x, y,y_pred)
    
    print(f'Los valores de log_alpha = {log_alpha}, log_betha = {log_betha}')
    print(f'Los valores de alpha = {alpha}, betha = {betha}')

In [ ]:
x = list_days[70:]
y = ys[0][70:]

exponencial_regression(x,y)

Aquí ustedes se preguntaran "¡¿pero qué pasó?!" con algo de miedo, y yo les responderé "Descuida,no rompiste el modelo, solo confirmaste que ese modelo no es". Sí, así es como se ve cuando la curva de un modelo exponencial, cuando los datos originales no tienen un comportamiento exponencial. ¿Por qué pasa? Por que si tuvieramos una pantalla del tamaño de una pared, donde mostrar los datos, sin hacer zoom veríamos que hay más tendencias dentro de la línea. No te preocupes, acabas de resolver un problema enorme: resolver si es exponencial o no. Te ganaste una cerveza.

Ahora, ¿cuál es el modelo matemático que encaja? Pues el siguiente.

## Regresión polinomial

¿Se acuerdan de sus amigos que decían orgullosamente "otro feliz día sin usar el Trinomio Cuadrado Perfecto"? Bueno, hoy no es ese día. Descuida, no lo use... Para este caso en específico. La verdad, es que el Trinomio Cuadrado Perfecto, y las otras herramientas que viste en las clases de álgebra y cálculo, son muy usadas para predecir modelos. Tanto, que ya vienen dentro del Kernel de algunas librerías "como es este caso".

Tal vez te estes pensando "ajá, ¿ me dirás en qué lo usaste?" y con gusto responderé "Pues en el código. ¿Dónde más?". El mismo procedimiento usado para una ecuación de segundo grado es el que use aquí. ¿Qué por qué esa? Por qué ya vimos que la de primer grado no encajaba. (Dejaré un momento de suspenso) Sí, la de primer grado es la regresión lineal simple, la primera que les mostré. La de segundo grado es el equivalente a decirle al programa "Oye, hay una parte dónde cambia de dirección. No sé dónde pero considérala por fa. Grax, tqm".

In [ ]:
def solve_for_m_2(x, y):
    if type(x[0]) == datetime:
        x = map(datetime.toordinal, x)
        x = list(x)

    #SUM X
    n = len(x)
    sum_x = sum(x)
    sum_x2 = sum([x[i]**2 for i in range(len(x))])
    sum_x3 = sum([x[i]**3 for i in range(len(x))])
    sum_x4 = sum([x[i]**4 for i in range(len(x))])

    #SUM Y
    sum_y = sum(y)
    sum_xy = sum([x[i]*y[i] for i in range(len(x))])
    sum_x2y = sum([x[i]**2*y[i] for i in range(len(x))])

    #Matrix row
    r1 = [n , sum_x, sum_x2]
    r2 = [sum_x, sum_x2, sum_x3]
    r3 = [sum_x2, sum_x3, sum_x4]

    A = np.matrix([r1, r2, r3], dtype='float')
    b = np.matrix([sum_y, sum_xy, sum_x2y], dtype='float').reshape(-1,1)
    
    A_prime = np.linalg.solve(A,b)

    return A_prime

def polinomial_regression(x, y):

    if type(x[0]) == datetime:
        x_m = [n for n in range(len(x))]
    else:
        x_m = x

    A_prime = solve_for_m_2(x_m, y)
    a0 = A_prime[0,0]
    a1 = A_prime[1,0]
    a2 = A_prime[2,0]

    y_pred_pol = [(a0 + a1*x_m[i] + a2*x_m[i]**2) for i in range(len(x_m))]

    #Graficación
    plot_pol_regression(x, y, y_pred_pol)

def plot_pol_regression(x, y, y_pred, x_labels=None, y_labels=None, titles=None):
    
    fig, ax = plt.subplots(figsize=(7,5))
    plt.style.use('ggplot')
    
    data_sctr = ax.scatter(x, y, label='Data', color = 'g', marker='o', s=10)
    mdl_plt = ax.plot(x, y_pred, label='Model',marker='o', markersize=1 , color = 'r', alpha=0.2, linewidth=10)
    #X axis format
    date_format = mdates.DateFormatter('%m')
    ax.set_xlabel('Dates')
    ax.xaxis.set_major_formatter(date_format)

    #Labels
    if x_labels == None:
        ax.set_xlabel('2020 months')
    else:
        pass

    if y_labels == None:  
        ax.set_ylabel('Confirmed Cases / 10,000,000')
    else:
        pass

    if titles == None: 
        ax.set_title('COVID-19 Confirmed Cases')
    
    plt.legend()

    plt.show()

In [ ]:
x = list_days[70:]
y = ys[0][70:]

polinomial_regression(x,y)
print('\n')

x = list_days
y = ys[0]

polinomial_regression(x,y)


Y ¡Oh, sorpresa! Encajó en ambos casos (Datos originales y cortados). Afortunadamente, por esto, no es necesario agregarle más grados a la regresión y damos por encontrado nuestro modelo. ¿Que por qué no le agregamos más? ¡Oh! Fácil, por qué buscamos que el modelo sea igual lo suficientmente general cómo para agregar más datos. No buscamos que copie los resultados en sí, sino su comportamiento, su tendencia pues. 

¿Todo claro? Excelente, vamos a las jugosas predicciones.

# Predicción Mundial con el modelo

¿Sientes una confusión de cómo se predijo? No te preocupes, ahorita te explico. Como el modelo encaja, no obtuvimos datos sintéticos (en sí), si no que generamos una ecuación a la que le podemos insertar más fechas (datos) para ver que pasará a futuro (extrapolar la curva). ¿Qué tanto? Tanto como queramos en sí, para este caso, agregaré tres meses para que se vea mayor el impacto de que ocurriría en el mundo si seguimos con el mismo comportamiento.

In [ ]:
year = end_day.year
month = end_day.month + 3
pred_month = calendar.month_name[month]

day = end_day.day

prediction_day = date(year, month, day)
start_day = date(year=2020, month=1, day=22)

prediction_days = prediction_day - start_day
n_prediction_days = prediction_days.days

list_days_pred = [start_day + timedelta(days=x) for x in range(n_prediction_days)]

In [ ]:
def polinomial_prediction(x, y, x_prediction):

    x_m = [n for n in range(len(x))]

    A_prime = solve_for_m_2(x_m, y)
    a0 = A_prime[0,0]
    a1 = A_prime[1,0]
    a2 = A_prime[2,0]

    x_m_p = [n for n in range(len(x_prediction))]

    y_pred_pol = [(a0 + a1*x_m_p[i] + a2*x_m_p[i]**2) for i in range(len(x_m_p))]

    return y_pred_pol

def plot_prediction(x, y, y_pred, x_pred, x_labels=None, y_labels=None, titles=None):
    
    fig, ax = plt.subplots(figsize=(10,6))

    mdl_plt = ax.plot(x_pred, y_pred, label='Model',marker='o', markersize=1 , color = 'r', alpha=0.2, linewidth=10)
    data_sctr = ax.scatter(x, y, label='Data', color = 'g', marker='o', s=10)

    #Labels
    ax.set_ylabel('Confirmed Cases / 10,000,000')
    ax.set_title('COVID-19 Confirmed Cases')
    
    pred_number = '{:,}'.format(int(y_pred[-1]))
    pred_date = datetime.strftime(x_pred[-1], '%d-%B-%Y')
    
    #X axis format
    txt = f'''2020 Months

Pronósitco: {pred_number}* Para el día: {pred_date}
*En caso de no haber una vacuna'''

    date_format = mdates.DateFormatter('%m')
    ax.xaxis.set_major_formatter(date_format)
    ax.set_xlabel(txt)

    plt.legend(loc='best')

    plt.show()

In [ ]:
def plot_cases(x, ys, y_pred, x_pred, labels, colors, country=None):
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5), sharey=False, sharex=True)
    n = 0
    plt.style.use('ggplot')

    for ax in fig.get_axes():
        mdl_plt = ax.plot(x_pred, y_pred[n], label='Model',marker='o', markersize=1 , color = 'r', alpha=0.2, linewidth=10)
        data_sctr = ax.scatter(x, ys[n], label='Data', color = colors[n], marker='o', s=2)
        
        ax.set_ylabel(f'{labels[n]} Cases / 1e^n')
        if country == None:
            ax.set_title(f'COVID-19 {labels[n]} World Cases')
        else:
            ax.set_title(f'COVID-19 {labels[n]} {country} Cases ')

        pred_number = '{:,}'.format(int(y_pred[n][-1]))
        pred_date = datetime.strftime(x_pred[-1], '%d-%B-%Y')

        if country == None:
            txt = f'''2020 Months

    World Prediction: {pred_number}* {labels[n]} at: {pred_date}
    *In case there is no vaccine'''

        else:
            txt = f'''2020 Months

    {country} Prediction: {pred_number}* {labels[n]} at: {pred_date}
    *In case there is no vaccine'''

        date_format = mdates.DateFormatter('%m')
        ax.xaxis.set_major_formatter(date_format)
        ax.set_xlabel(txt)

        mdl_plt = mpatches.Patch(color='red', label='Model', alpha=0.2)
        ax.legend([mdl_plt, data_sctr], ['Model', 'Data'], loc='best')

        n = n + 1

    plt.show()

In [ ]:
x = list_days
ys = [sum_confirmed, sum_deaths, sum_recovered]

x_pred = list_days_pred
y_pred = [polinomial_prediction(x, ys[n], x_pred) for n in range(len(ys))]

labels = ['Confirmed', 'Deaths', 'Recovered']

plot_cases(x, ys, y_pred, x_pred, labels, colors)

Como pueden ver, si a nivel mundial seguimos el mismo comportamiento, las muertes mundiales podrían incrementar hasta el millón y medio para noviembre. Y porqué lo quiero volver personal, haré el análisis para mi país.

# Mexico

Mi México lindo y querido (8), tiene una fama de no querer invertir mucho en investigación y tecnología, pero esto es por qué no ha visto sus bondades. Si supieran que cuando resuelves el problema para un caso, lo resuelves para todos (es un dicho, no una verdad, cálmense). Para este trabajo (simple), así fue. Si sabes de programación, te invito a comparar la diferencia entre el código de antes y el de ahora. Verás que a lo largo de todo este camino, solo hice herramientas que reusaba. Afortunadamente, para el modelo de México funcionó.

In [ ]:
country = df['Country/Region'].to_list()

mx_confirmed = [0] * (n_days)
mx_deaths = [0] * (n_days)
mx_recovered = [0] * (n_days)

day_subregion = [None] * (n_days)
day_continent = [None] * (n_days)

plus = 0
for d in range(len(list_days)):

    while dates[plus] == list_days[d] and (d + plus) < len(dates):
        '''---BY DAY---'''
        #General
        if country[d + plus] == 'Mexico':
            mx_confirmed[d] = int(mx_confirmed[d]) + int(confirmed[d + plus])
            mx_deaths[d] = int(mx_deaths[d]) + int(deaths[d + plus])
            mx_recovered[d] = int(mx_recovered[d]) + int(recovered[d + plus])
        
        plus += 1

In [ ]:
x = list_days[70:]
y_mx = [mx_confirmed[70:], mx_deaths[70:], mx_recovered[70:]]

x_pred = list_days_pred[70:]
y_pred_mx = [polinomial_prediction(x, y_mx[n], x_pred) for n in range(len(y_mx))]

plot_cases(x, y_mx, y_pred_mx, x_pred, labels, colors, 'Mexico')

# Conclusiones

Es curioso como una herramienta que ví en la prepa, la cuál, no entendí en un inicio, me ayudó en un reto de hoy. Y no solo a mí, sino a muchos de ustedes, solo que no nos dimos cuenta. ¿Quién no ha pensado a futuro usando sus ingresos como parámetro? Digo, siempre los queremos aumentar, ¿verdad?

Lo que me hace pensar, tal vez las matemáticas no sean aburridas, sino que nos contaron mal una historia. Si me hubiesen contado que de saber esto podría ayudar a alguien con su trabajo, o que con esto podría diseñar un modelo para ayudar a mi comunidad, con ejemplos reales, tal vez, la historisa sería distinta.

Por todo esto, te dejo una invitación: vuelve a ser un infante curioso. Vuelve a preguntar el por qué de todo, cómo funciona. Puede que te cuenten otra vez una historia de superación personal, y cuando te des cuenta, veas en la portada tu foto. Se vale soñar, pero es bello tener la cabeza en las nubes y los pies en el suelo.

# Epílogo. Seamos serios un momento

Cuando me estaba alegrando por haber cumplido el reto, ya destapando mi cervecita, recuerdo que estaba usando información real, y volví a ver los datos. Sin ser videntes, usando matemáticas, pudimos hechar un vistazo al futuro lejano. Probé con uno cercano y trágicamente, se acercó, con un error de +/- 11% de error por caso.

Tal vez digas: "a bueno, si el error se acumula quiere decir conforme pasa el tiempo esa cosa se equivocará más, o sea que no llegaremos a tanto". Y sí... pero no. Recuerda que el error es tanto para arriba, como para abajo, para darle un rango de tolerancia. No buscamos "copiar los datos" sino analizar su comportamiento. Quiere decir que a cómo hoy falló por que no se llegó a la cifra, a cómo mañana puede que se acumulen y "alcancemos la cuota" para seguir la misma tendencia. Estas cifras **reales** sobre muertes **registradas POR DÍA** por COVID me pueden ayudar a explicar mejor:

* 02 de junio: 470
* 03 de junio: 1,092


* 09 de agosto: 292
* 11 de agosto: 926


* 20 de agosto: 625

¿Qué quiero mostrar? Que en uno o dos días podemos triplicamos las muertes por COVID. No bajen la guardia, por que quiere decir que un día los periódicos anunciar que perdimos 2,000 paisanos. Tal vez algunos fueron irresponsables, pero... ¿Y los que no? Cuándo creas que no ya pasamos lo peor, recuerda, lo peor no acaba hasta que pasen 3 semanas con 0 muertes. Mientras tanto les invito a quedarse en casa, respetar las normas sanitarias, no hacer fiestas y nunca parar de aprender.

Al menos por hoy, ya entendiste por qué es importante quedarse en casa. Y sí, estas son las bases para predecir acciones en la bolsa o en criptos, aunque necesitas saber un poco (mucho) más, todo lo podrás encontrar en google y gratis (la vía difícil) y en cursos en línea.

## Anexo: Mexico vs The World

In [ ]:
x = list_days[70:]
y_mx = [mx_confirmed[70:], mx_deaths[70:], mx_recovered[70:]]

x_pred = list_days_pred[70:]
ys = [sum_confirmed[70:], sum_deaths[70:], sum_recovered[70:]]
y_pred = [polinomial_prediction(x, ys[n], x_pred) for n in range(len(ys))]
y_pred_mx = [polinomial_prediction(x, y_mx[n], x_pred) for n in range(len(y_mx))]

labels = ['Confirmed', 'Deaths', 'Recovered']
mx_colors = [bmh_colors['blue'], 'k', bmh_colors['green']]

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5), sharey=False, sharex=True)
n = 0
plt.style.use('ggplot')

for ax in fig.get_axes():
    world_mld_plt = ax.plot(x_pred, y_pred[n], label='World Model',marker='o', markersize=1 , color = 'r', alpha=0.2, linewidth=10)
    mx_mdl_plt = ax.plot(x_pred, y_pred_mx[n], label='Mx Model',marker='o', markersize=1 , color = bmh_colors['orange'], alpha=0.2, linewidth=10)

    data_sctr = ax.scatter(x, ys[n], label='World Data', color = colors[n], marker='o', s=2)
    mx_sctr = ax.scatter(x, y_mx[n], label='Mx Data', color = mx_colors[n], marker='o', s=2)
    
    ax.set_ylabel(f'{labels[n]} Cases / 1e^n')
    ax.set_title(f'COVID-19 {labels[n]} Cases')

    pred_number = '{:,}'.format(int(y_pred[n][-1]))
    pred_date = datetime.strftime(x_pred[-1], '%d-%B-%Y')

    pred_number_mx = '{:,}'.format(int(y_pred_mx[n][-1]))

    txt = f'''2020 Months

World Prediction: {pred_number}* {labels[n]}
Mx Prediction:  {pred_number_mx}* {labels[n]}
At: {pred_date}

*In case there is no vaccine'''

    date_format = mdates.DateFormatter('%m')
    ax.xaxis.set_major_formatter(date_format)
    ax.set_xlabel(txt)


    n = n + 1
    
    ax.legend(loc='best')

plt.show()